ref : 

https://colab.research.google.com/drive/1YrmsVYrzcvPaWht-yXPbqthiJvHf78qB?usp=sharing%EF%BB%BF

https://python.langchain.com/docs/use_cases/chatbots/

In [1]:
# OpenAI API Key 
# 로컬 모델은 일단 너무 느린 듯....

from dotenv import load_dotenv
import os

load_dotenv('../.env')

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [3]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.chat_models import ChatOpenAI

# default model : gpt-3.5-turbo-0125
chat = ChatOpenAI(openai_api_key = OPENAI_API_KEY)

In [4]:
# chat test 
chat(
    [
        HumanMessage(
            content="Who are you?"
        )
    ]
)

AIMessage(content='I am a language model AI developed by OpenAI. How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 11, 'total_tokens': 29}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-d6ac636a-64fb-4719-aa8c-6d1d4effc9dd-0')

In [5]:
# conversation chain : 사용자 입력과 모델 출력값을 history로 갖는 메모리 체인 모듈 
from langchain.chains import ConversationChain

conversation = ConversationChain(llm = chat)
conversation.run("다음 문장을 한국어로 번역하시오. I love you.")

'저는 당신을 사랑합니다.'

In [6]:
conversation.run("일본어로 번역하시오.")

'愛しています。'

In [8]:
# conversation buffer memory : 모든 대화 내용을 기록하는 모듈 
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hello.")
memory.chat_memory.add_ai_message("hello, sir.")

memory.load_memory_variables({})

{'history': 'Human: hello.\nAI: hello, sir.'}

In [9]:
# conversation buffer window memory : 가장 최근 k개 대화 내역 저장 
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=2)
memory.save_context({"input": "hello"}, {"output": "hello, sir."})
memory.save_context({"input": "how are you?"}, {"output": "I'm fine."})
memory.save_context({"input": "how are you, too?"}, {"output": "I'm fine, too."})

memory.load_memory_variables({})

{'history': "Human: how are you?\nAI: I'm fine.\nHuman: how are you, too?\nAI: I'm fine, too."}

Conversation : 대화형 chatbot 만들기 

Memory + prompt + llm 의 구조

In [10]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chains import LLMChain

# llm 설정 
# open ai 기본 모델 : gpt-3.5-turbo
llm = ChatOpenAI(openai_api_key = OPENAI_API_KEY)

memory_key = "chat_history"

# prompt 지정
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are chatbot for human."
        ),
        MessagesPlaceholder(variable_name=memory_key), # variable_name : memory 연결 위한 key 
        HumanMessagePromptTemplate.from_template("{question}"),
    ]
)

memory = ConversationBufferMemory(memory_key=memory_key, return_messages=True)
conversation = LLMChain(llm = llm, prompt = prompt, verbose = True, memory = memory)

In [11]:
conversation({"question" : "hello"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are chatbot for human.
Human: hello

> Finished chain.


{'question': 'hello',
 'chat_history': [HumanMessage(content='hello'),
  AIMessage(content='Hello! How can I assist you today?')],
 'text': 'Hello! How can I assist you today?'}

In [12]:
# memory와 retriver 결합한 chatbot

# pdf 파일 가져오기
from langchain.document_loaders import PyPDFLoader

# url을 통해서 pdf 정보 가져오기
loader = PyPDFLoader("https://snuac.snu.ac.kr/2015_snuac/wp-content/uploads/2015/07/asiabrief_3-26.pdf")
pages = loader.load_and_split()

In [13]:
# PDF 내용을 작은 chunk 단위로 나누기
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(pages)

In [14]:
# vector store에 저장

from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

# text-embedding-ada-002
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY))

In [15]:
# conversation retriver chain 이용해서 memory + retriever + llm 결합

from langchain.chains import ConversationalRetrievalChain

retriever = vectorstore.as_retriever()
qa = ConversationalRetrievalChain.from_llm(llm, retriever = retriever, memory = memory)

In [16]:
qa("저출산을 극복한 나라들은 어디야?")

{'question': '저출산을 극복한 나라들은 어디야?',
 'chat_history': [HumanMessage(content='hello'),
  AIMessage(content='Hello! How can I assist you today?'),
  HumanMessage(content='저출산을 극복한 나라들은 어디야?'),
  AIMessage(content='중 프랑스, 스웨덴, 영국은 저출산 현상을 극복한 국가들로 알려져 있습니다.')],
 'answer': '중 프랑스, 스웨덴, 영국은 저출산 현상을 극복한 국가들로 알려져 있습니다.'}

In [17]:
qa("스웨덴은 어떻게 저출산 현상을 극복했어?")

{'question': '스웨덴은 어떻게 저출산 현상을 극복했어?',
 'chat_history': [HumanMessage(content='hello'),
  AIMessage(content='Hello! How can I assist you today?'),
  HumanMessage(content='저출산을 극복한 나라들은 어디야?'),
  AIMessage(content='중 프랑스, 스웨덴, 영국은 저출산 현상을 극복한 국가들로 알려져 있습니다.'),
  HumanMessage(content='스웨덴은 어떻게 저출산 현상을 극복했어?'),
  AIMessage(content='스웨덴은 저출산 현상을 극복하기 위해 여성 고용률을 높이는 등의 다양한 정책을 시행했습니다. 이를 위해 노동시장에서의 젠더 차별을 없애고, 일가정양립 환경을 조성하는 등의 노력을 통해 여성이 일과 가정 생활을 양립할 수 있는 환경을 조성했습니다. 또한, 고령화가 진행됨에 따라 고령인구를 노동시장에 더 오랫동안 남도록 하는 정년 연장 등의 정책도 시행했습니다. 이러한 노력들이 저출산 대책에 기여했을 것으로 보입니다.')],
 'answer': '스웨덴은 저출산 현상을 극복하기 위해 여성 고용률을 높이는 등의 다양한 정책을 시행했습니다. 이를 위해 노동시장에서의 젠더 차별을 없애고, 일가정양립 환경을 조성하는 등의 노력을 통해 여성이 일과 가정 생활을 양립할 수 있는 환경을 조성했습니다. 또한, 고령화가 진행됨에 따라 고령인구를 노동시장에 더 오랫동안 남도록 하는 정년 연장 등의 정책도 시행했습니다. 이러한 노력들이 저출산 대책에 기여했을 것으로 보입니다.'}